In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime,timedelta
import json
import re

In [2]:
#根据log_index，增量读取log内容
def load_log(file_name, log_index):
    log_value_list = list()
    date_list = list()
    status_list = list()
    try:
        with open(file_name,'rb') as f:
            for index,content in enumerate(f):
                if index == log_index:
                    try:
                        log_data = json.loads(content.decode('utf8'))
                    except Exception as e:
                        print ('json解析错误',index,content[:50])
                        log_index += 1
                        continue
                        
                    if '@timestamp' in log_data:
                        log_data['@timestamp'] = str(datetime.strptime(log_data['@timestamp'][0:19],'%Y-%m-%dT%H:%M:%S'))
                        log_value_list.append(log_data)
                        date_list.append(log_data['@timestamp'])
                        if log_data['status'] not in status_list:
                            status_list.append(log_data['status'])
                    log_index += 1

    except Exception as e:
        print('其他错误',index,e)
    #将文本内容转化为pandas的dataframe
    #temp_log = pd.DataFrame(log_value_list)
    print(status_list)
    return  log_value_list,log_index

In [20]:
num_urls_in_op = 20 #关键参数，每x个url作为一组进行分配
data_type = 'test'  #test or train

In [5]:
time_step = 120 #每x秒读取一次log文件
time_windows = 120 #秒,时间窗口
ip_dict = dict() #ip与对应操作在矩阵的下标
ip_reserve_list = list() #操作矩阵下标对应的ip
all_matrix = np.empty([0, 6],dtype=int)  #操作矩阵
all_log = pd.DataFrame() #所有的日志数据
temp_log_data = pd.DataFrame() #某时间范围的日志数据
all_url_list = list()
time_index = '2017-07-13 06:25:08' #记录当前的时间
log_index = 0 #记录读到第几行了
ip_behavior_dict = dict() #记录每个ip每分钟请求的页面url

increment_log_data, log_index = load_log('data/'+data_type+'_access_szy_shunnengnet_com_json.log', 0)

['200', '302', '404', '504', '403', '304', '500']


In [6]:
len(increment_log_data)

32766

In [7]:
#截取url，保留其访问传参方式（POST/GET/PUT/DELETE）,保留url的头与尾
def deal_url(url):
    url_str = url.replace(' HTTP/1.1','')
    if url_str.find(' ') > -1:
        url_type = url_str[0 : url_str.find(' ')+1]
    if url_str.find('/') > -1 and url_str.find('?') > -1:
        url_str = url_type + url_str[url_str.find('/')+1 : url_str.find('?')]
    if url_str.find('/') > -1 and url_str.find(';jsessionid') > -1:
        url_str = url_type + url_str[url_str.find('/')+1 : url_str.find(';jsessionid')]
    if url_str.find('/') > -1 and url_str.rfind('/') > -1:
        url_str = url_str[0 : url_str.find('/')] + '/.../' + url_str[url_str.rfind('/')+1 : ]
    return url_str

#根据日志内容，创建一个信息的op_detail
def create_op_detail(row_log):
    op_detail = dict()
    op_detail['sip'] = row_log['sip']
    op_detail['start_time'] = row_log['@timestamp']
    op_detail['end_time'] = row_log['@timestamp']
    op_detail['urls'] = list()
    op_detail['urls'].append(deal_url(row_log['request']))
    return op_detail

In [8]:
#传入开始结束时间的字符串(yyyy-MM-dd HH:mm:ss)，计算时间差并返回时差(秒)
def cacl_time_difference(start_time_str, end_time_str):
    start_time = datetime.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_time = datetime.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    return int((end_time - start_time).seconds)

In [9]:
#读取log，生成携带用户行为的复合型字典
def create_user_behavior_dict(increment_log_data, user_behavior_dict):
    
    for x in increment_log_data:
        if x['sip'] != '':
            
            if x['sip'] not in user_behavior_dict.keys():
                user_behavior_dict[x['sip']] = list()

            if len(user_behavior_dict[x['sip']]) == 0: #opreation内没有op_detail,则创建op_detail
                user_behavior_dict[x['sip']].append(create_op_detail(x))
            else: #opreation内已存在op_detail，此时需要循环该list的所有dict，找到未填充满num_urls_in_op的记录
                new_opreation = 1
                i = 0
                for op_detail in user_behavior_dict[x['sip']]:
                    if len(op_detail['urls']) < num_urls_in_op:
                        op_detail['end_time'] = x['@timestamp']
                        op_detail['urls'].append(deal_url(x['request']))
                        new_opreation = 0
                        break
                if new_opreation == 1:
                    user_behavior_dict[x['sip']].append(create_op_detail(x))
                    i += 1
                    
                
    return user_behavior_dict
        
        

In [10]:
user_behavior_dict = dict()

In [11]:
start_time = datetime.now()
user_behavior_dict = create_user_behavior_dict(increment_log_data, user_behavior_dict)
end_time = datetime.now()
print((end_time - start_time).seconds)

0


In [14]:
c = 1
for x in increment_log_data:
    if x['sip'] == '':
        print (c)
        c+=1

1
2
3
4
5
6
7
8
9
10
11
12


In [15]:
sum = 0
for ip in user_behavior_dict:
    for op in user_behavior_dict[ip]:
        sum += len(op['urls'])
sum

32754

In [13]:
# #定义模板list
# templates = list()
# templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do']) #普通专家预约
# templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../expertNotice.do','POST /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#知名专家预约
# templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../specialClinic.do','POST /.../getDepts.do','GET /common/.../specialSchedule.do','POST /.../specialSchedule.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#特色门诊预约
# templates.append(['GET /common/.../patientinfo.do','GET /common/.../reserveList.do','POST /.../getReserveList.do','GET /common/.../reserveDetail.do','POST /.../cancelReserve.do'])#取消预约
# templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /.../patientinfo.do','GET /common/.../addPatient.do','POST /.../checkIdCard.do','POST /.../create.do'])#新增就诊人
# templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','POST /.../setDefault.do',''])#设置默认就诊人
# templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /common/.../bind.do','POST /.../bindHosCard.do'])#绑卡
# templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /.../deletePatient.do'])#删除就诊人
# templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportList.do','GET /common/.../reportDetail.do','GET /.../getReportDeatil.do'])#门诊报告
# templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportInList.do','GET /.../getReportDeatil.do'])#住院报告
# templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../balance.do','GET /.../patientBalance.do'])#门诊预交金
# templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../hosBalance.do','POST /.../inBalance.do'])#住院预交金
# templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../outpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#门诊费用
# templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../inpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#住院费用
# templates.append(['GET /.../init.do','GET /common/.../reserveQueueList.do','POST /.../getQueueList.do'])#候诊队列
# templates.append(['POST coreServlet'])#处理微信发送来的数据
# templates.append(['GET common/.../arrangementList.do', 'GET /.../404.do','POST /.../getTimeList.do'])#异常模板1
# #templates.append(['POST /.../getTimeList.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','GET /.../404.do','GET common/.../arrangementList.do','GET /.../404.do'])#异常模板1
# templates.append(['GET common/.../scheduleByDept.do','POST /.../scheduleByDept.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do','POST /.../getTimeList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板2
# #templates.append(['GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板3
# templates.append(['GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do'])

In [14]:
# probability_dict = dict()

# for x in user_behavior_dict.keys():
#     for op_detail in user_behavior_dict[x]:
#         matched_operations = np.zeros(len(templates),dtype='float32')
#         for idx,template in enumerate(templates):
#             n_template = len(template)
#             count = 0
#             probability_count = 0
#             for str in template:
#                 if str in op_detail['urls']:
#                     probability_count += 1
#             for url in op_detail['urls']:
#                 if url in template:
#                     count += 1
#             n_op = max(len(op_detail['urls']),16)
#             matched_operations[idx] = (probability_count / n_template) * (count / n_op)
#         if x not in probability_dict.keys():
#             probability_dict[x] = matched_operations.reshape([1,19])
#         else:
#             probability_dict[x] = np.vstack((probability_dict[x],matched_operations))
        
        

In [16]:
idx = 0
for x in user_behavior_dict.keys():
    idx += len(user_behavior_dict[x])
idx

2824

In [17]:
import time
#速度变慢的罪魁祸首
def judge_time_range(start_time_str, end_time_str, time_str):
    start_date = time.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_date = time.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    time_date = time.strptime(time_str,'%Y-%m-%d %H:%M:%S')
    
    if time_date > start_date and time_date < end_date:
        return True
    else:
        return False
    

In [18]:
#!!!生成出现过的ip列表unique_dict
#为每一个ip计算他访问过多少唯一页面，占访问次数的比例。frequency这英文似乎给错了，不是频率，而是覆盖率coverage 


unique_dict = dict()
for x in increment_log_data:
    ip = x['sip']
    if ip not in unique_dict.keys():
        unique_dict[ip] = dict()
        unique_dict[ip]['coverage'] = 0
        unique_dict[ip]['urls'] = list()
    url = deal_url(x['request'])
    unique_dict[ip]['urls'].append(url)
    
for ip in unique_dict.keys():
    unique_list = list()
    for url in unique_dict[ip]['urls']:
        if url not in unique_list:
            unique_list.append(url)
    #print(len(unique_dict[ip]['urls']))
    unique_dict[ip]['coverage'] = len(unique_list) / len(unique_dict[ip]['urls'])

In [21]:
# def get_operation_label(ip, time):
#     global probability_dict
#     class_type = 0
#     for x in user_behavior_dict:
#         if x == ip:
#             index = 0
#             for op_detail in user_behavior_dict[x]:
#                 start_str = op_detail['start_time']
#                 end_str = op_detail['end_time']
#                 if judge_time_range(start_str, end_str, time):
#                     class_type = probability_dict[ip][:,16:].sum()
#                     #if probability_dict[ip][:,16:].sum() > 1:
#                         #class_type = 1
#                     #elif probability_dict[ip][index,17] > 0.4:
#                         #class_type = 1
#                    # elif probability_dict[ip][index,18] > 0.4:
#                         #class_type = 1
#                     break #如果匹配到了，也给了class_type了，还继续循环干什么
#                 else:
#                     index += 1
#     return class_type


# def get_operation_label(ip):
#     global unique_dict
#     class_type = 0
#     if unique_dict[ip]['coverage'] < 0.1:
#         class_type = 1
#     return class_type

import json
with open(data_type+'_label_json.txt', 'r') as f:
    labels_dict = json.loads(f.read())
def get_operation_label(ip):
    global labels_dict
    return labels_dict[ip]['label']

FileNotFoundError: [Errno 2] No such file or directory: 'test_label_json.txt'

In [19]:
exception_ip_list = list()
for x in unique_dict.keys():
    if unique_dict[x]['coverage'] < 0.15:
        exception_ip_list.append(x)
len(exception_ip_list)/len(unique_dict)

0.08907346412716598

In [20]:
get_operation_label('112.224.2.20')

1

In [21]:
#----------------------start
class IPAccessInfo(object):
    def __init__(self, all_url_list, all_url_dict_list):
        self.all_url_list = all_url_list
        self.all_url_dict_list = all_url_dict_list
        
import pickle
fp = open('data/url_info.pkl','rb',True)
iPAccessInfo = pickle.load(fp)

In [22]:
# from keras.models import Sequential
# from keras.layers import Embedding

# #列出唯一的url
# url_dict = dict()
# for urls in iPAccessInfo.all_url_list:
#     for url in urls:
#         if url not in url_dict.keys():
#             url_dict[url] = 1
#         else:
#             url_dict[url] = url_dict[url] + 1
            
# url_list = list(url_dict.keys())

# #将url embedding，转为固定长度的向量
# num_urls = len(url_list)
# model = Sequential()
# model.add(Embedding(num_urls, 32, input_length=1))
# model.compile('rmsprop', 'mse')

# input_array = np.array(range(num_urls))   #生成一个[num_url,1]的矩阵
# output_array = model.predict(input_array)
# print (output_array.shape)
# print (output_array)

# # 建立url_dict备查
# url_dict = dict()
# idx = 0
# for url in url_list:
#     url_dict[url] = output_array[idx,0,:]
#     idx = idx + 1


import pickle
fp = open('url_embedding.pkl','rb',True)
url_dict = pickle.load(fp)


In [23]:
#老版本数据转换，将按照时间窗口、ip进行归类的urls列表，转化为时序data
# def operations_to_sequence(all_url_dict_list):
#     operation_matrix = np.zeros((len(all_url_dict_list),20,32))
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         ip_operation_url_index = 0
#         for url in ip_operations['urls']: #如果是dict字典的话
#             url_vector = url_dict[url]
#             operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
#             ip_operation_url_index = ip_operation_url_index + 1
#             if ip_operation_url_index>=20:  #不超过20个长度的序列，可以改
#                 break
#         ip_operation_index = ip_operation_index + 1
#     return operation_matrix


#新版本数据转换，将按照ip地址，然后里面的几段行为数据，转化为时序data
# 数据格式为：
# {'223.104.186.241': [{'end_time': '2017-07-13 06:25:32',
#    'start_time': '2017-07-13 06:25:02',
#    'urls': ['POST /.../getDept.do',
#     'POST /.../getDept.do',
#     'GET common/.../scheduleByDept.do',
#     'POST /.../scheduleByDept.do',
#     'GET common/.../arrangementList.do',
#     'GET common/.../arrangementList.do',
#     'POST /.../getTimeList.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'GET common/.../patientList.do',
#     'GET /.../patientinfo.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'POST /.../reserve.do']},
#   {'end_time': '2017-07-13 07:31:18',
#    'start_time': '2017-07-13 07:26:36',
#    'urls': ['GET /.../noticePage.html',

def operations_to_sequence(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
    
    operation_matrix = np.zeros((n_op_details,num_urls_in_op,32))
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_detail in user_behavior_dict[ip]: #对于每一组urls操作
            ip_operation_url_index = 0
            for url in op_detail['urls']: #对于每一个url
                url_vector = url_dict[url]  #翻译为url对应的vector
                operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
                ip_operation_url_index += 1
                if ip_operation_url_index>=num_urls_in_op:  #不超过20个长度的序列，可以改
                    break
            ip_operation_index += 1
    return operation_matrix

In [24]:
unique_url_list_count = dict() #计算一下每一个url出现的次数，url:count
for x in increment_log_data:
    ip = deal_url(x['request'])
    if ip not in unique_url_list_count:
        unique_url_list_count[ip] = 0
    unique_url_list_count[ip] +=1

#生成访问记录所有url的list，唯一，只有大于20的url才被认为具有普遍性
unique_url_list = [x for x in unique_url_list_count.keys() if unique_url_list_count[x] > 20]

In [25]:
unique_url_list_count

{'GET /.../': 347,
 'GET /.../coreServlet': 1,
 'GET /.../error303.png': 45,
 'GET /.../favicon.ico': 218,
 'GET /.../font_1447920314_865217.css': 1,
 'GET /.../index_network.html': 936,
 'GET /.../robots.txt': 1,
 'GET SN-SZY-Server/.../bindByAlipay.do': 32,
 'GET SN-SZY-Server/.../bindByNet.do': 17,
 'GET SN-SZY-Server/.../bindByWechat.do': 1167,
 'GET SN-SZY-Server/.../bindNotice.do': 358,
 'GET SN-SZY-Server/.../cancelRegister.do': 3,
 'GET SN-SZY-Server/.../checkDetail.do': 42,
 'GET SN-SZY-Server/.../checkList.do': 898,
 'GET SN-SZY-Server/.../delete.do': 2,
 'GET SN-SZY-Server/.../eastDept.do': 8551,
 'GET SN-SZY-Server/.../examineDetail.do': 2420,
 'GET SN-SZY-Server/.../examineList.do': 5180,
 'GET SN-SZY-Server/.../feeNotice.html': 123,
 'GET SN-SZY-Server/.../getRegResource.do': 21760,
 'GET SN-SZY-Server/.../head.html': 203,
 'GET SN-SZY-Server/.../inFeeDetail.do': 954,
 'GET SN-SZY-Server/.../inFeeList.do': 1371,
 'GET SN-SZY-Server/.../index_network.html': 1,
 'GET SN-SZY

In [26]:
data = operations_to_sequence(user_behavior_dict)
print (data.shape) #数据编码，我就是快，数据量大640倍，还是快。。。

(13385, 20, 32)


In [27]:
len(user_behavior_dict['27.204.15.164'][2]['urls'])

KeyError: '27.204.15.164'

In [28]:
# 旧版的数据转换
# def label_operations(all_url_dict_list):
#     operation_label = np.zeros(len(all_url_dict_list)) # 先按照0 1 2 3，一维数组排序
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         operation_label[ip_operation_index] = get_operation_label(ip_operations['ip'],ip_operations['time'])
#         ip_operation_index = ip_operation_index + 1
        
#     #转化为one-hot编码的label
#     return operation_label


def label_operations(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
        
    operation_labels = np.zeros(n_op_details) # 先按照0 1 2 3，一维数组排序
    operation_coverages = np.zeros(n_op_details) # 先按照0 1 2 3，一维数组排序
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_index in range(len(user_behavior_dict[ip])): #对于每一组urls操作，得到其index 
            operation_labels[ip_operation_index] = get_operation_label(ip)
            operation_coverages[ip_operation_index] = labels_dict[ip]['coverage']
            ip_operation_index += 1
            
    return operation_labels, operation_coverages

In [29]:
# tmd为啥这么慢，为啥这么慢，为啥这么慢啊!!!!!!
labels, coverages = label_operations(user_behavior_dict)
print (labels.shape)

(13385,)


In [30]:
(labels==1).sum()

3855

In [31]:
# #labels
# b = np.argsort(labels)
# b = b.tolist()
# b.reverse()
# for x in b:
#     if labels[x] > 0.9 and labels[x] < 1:
#         #ip = iPAccessInfo.all_url_dict_list[x]['ip']
#         print(iPAccessInfo.all_url_dict_list[x])
        
        

In [32]:
(labels==0).sum() #验证每个类别的个数

5361

In [33]:
#网址embedding表示，应该不需要进行正规化吧
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# #由于时序序列数据，是（？，时间长度，数据长度）的，所以应该执行一个reshape
# X = scaler.fit_transform(data.reshape([-1,20*32]))
# X = X.reshape([-1,20,32])

In [34]:
# 定义保存数据的类
import numpy as np

#定义类
class IPAccessData(object):
    def __init__(self, X, y, coverage):
        self.X = X
        self.y = y
        self.coverage = coverage
#         self.ip_dict = ip_dict
#         self.ip_reserve_list = ip_reserve_list
#         self.data_scaler = data_scaler

# 保存文件为pkl
import pickle

iPAccessData = IPAccessData(data,labels,coverages)
fp = open('iPAccessData_count.pkl','wb',True)
pickle.dump(iPAccessData, fp)

In [35]:
def testFun(idx):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
    
    operation_matrix = np.zeros((n_op_details,num_urls_in_op,32))
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_detail in user_behavior_dict[ip]: #对于每一组urls操作
            if ip_operation_index == idx:
                print (op_detail)
                print (coverages[ip_operation_index])
            ip_operation_index += 1

In [54]:
testFun(400)

{'sip': '140.207.54.80', 'start_time': '2017-07-13 07:30:08', 'end_time': '2017-07-13 07:32:16', 'urls': ['POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet', 'POST SN-SZY-Server/.../coreServlet']}
0.000273224043715


In [37]:
testFun(1595)

{'sip': '112.235.108.133', 'start_time': '2017-07-13 07:24:02', 'end_time': '2017-07-13 07:24:14', 'urls': ['GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../init.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../initNotice.do', 'GET SN-SZY-Server/.../init.do', 'GET SN-SZY-Server/.../westDept.do', 'GET SN-SZY-Server/.../getRegResource.do', 'GET SN-SZY-Server/.../getRegResource.do']}
0.012224938875


In [38]:
get_operation_label('117.136.9.3')

2

In [55]:
user_behavior_dict['140.207.54.80']

[{'end_time': '2017-07-13 06:47:21',
  'sip': '140.207.54.80',
  'start_time': '2017-07-13 06:25:21',
  'urls': ['POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet',
   'POST SN-SZY-Server/.../coreServlet']},
 {'end_time': '2017-07-13 06:56:03',
  'sip': '140.207.54.80',
  'st

In [40]:
'119.189.123.131' in bad

NameError: name 'bad' is not defined